- Data su zo stranky www.soga.sk, jedna sa o vsetky aukcie od roku 2004 az po 2024, cca 100 aukcii a 21,000 drazenych diel

In [22]:
import sqlite3
import pandas as pd
import pandasql as ps
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.io import output_notebook

In [23]:
# Connect to the SQLite database
conn = sqlite3.connect('soga.db')

# Query to select data from the database
query = """
SELECT 
    UPPER(SUBSTR(autor, 1, INSTR(autor, '(') - 1)) AS autor,
    a.nazov, 
    vydrazene,
    CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) AS konecna_cena,
    CAST(REPLACE(vyvolavacia_cena, ' ', '') AS INTEGER) AS vyvolavacia_cena,
    rok,
    technika,
    datum,
    url_soga
FROM diela a 
LEFT JOIN aukcie b ON a.id_aukcie = b.id 
WHERE TRUE 
-- AND autor like '%FILKO%'
ORDER BY CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) DESC;
"""

# Load data into a Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
df.head()

,autor,nazov,vydrazene,konecna_cena,vyvolavacia_cena,rok,technika,datum,url_soga
0,BENKA MARTIN,Z Detvy,1,146000.0,NaN,1932,Olej na plátne,"10. November 2015, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...
1,MAJERNÍK CYPRIÁN,KRASOJAZDKYNE,1,125000.0,NaN,1938,Gvaš na papieri,"03. Jún 2025, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...
2,JAN HÁLA,Nad Važcom,1,112000.0,NaN,1943,Olej na plátne,"06. Jún 2023, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...
3,FULLA ĽUDOVÍT,Ukrižovanie,1,106500.0,78000.0,okolo 1939-42,olej na kartóne,05. Október 2010,https://www.soga.sk/aukcie-obrazy-diela-umenie...
4,FOLTÝN FRANTIŠEK,Dvaja,1,93000.0,NaN,1925,Olej na plátne,"13. November 2012, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...


# Priprava dat

In [24]:
from unidecode import unidecode  # Import the unidecode function
import re


# Custom function to standardize and reorder names
def standardize_and_order_name(name):
    # Convert accented characters to ASCII equivalents
    name = unidecode(name)
    name= name.upper()

    # Split the name into parts
    name_parts = name.split()

    # Reorder the name parts alphabetically
    standardized_name = ' '.join(sorted(name_parts))
    
    

    return standardized_name


def extract_first_year(date_string):
    # Regular expression pattern to match a year
    year_pattern = re.compile(r'\b\d{4}\b')

    # Find the first match in the string
    match = year_pattern.search(date_string)

    # Extract and return the matched year
    if match:
        return int(match.group())

    return None  # Return None if no year is found


In [25]:
df['autor']=df['autor'].apply(standardize_and_order_name)

In [26]:
df['olej']=df['technika'].apply(lambda x: 'olej' in x.lower())

In [27]:
df['rok_vytvorenia']=df['rok'].apply(lambda x: extract_first_year(x)).fillna(-1).astype(int).replace(-1,None)
df['rok_drazby']=df['datum'].apply(lambda x: extract_first_year(x)).fillna(-1).astype(int).replace(-1,None)

In [28]:
df.head()

,autor,nazov,vydrazene,konecna_cena,vyvolavacia_cena,rok,technika,datum,url_soga,olej,rok_vytvorenia,rok_drazby
0,BENKA MARTIN,Z Detvy,1,146000.0,NaN,1932,Olej na plátne,"10. November 2015, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...,True,1932,2015
1,CYPRIAN MAJERNIK,KRASOJAZDKYNE,1,125000.0,NaN,1938,Gvaš na papieri,"03. Jún 2025, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...,False,1938,2025
2,HALA JAN,Nad Važcom,1,112000.0,NaN,1943,Olej na plátne,"06. Jún 2023, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...,True,1943,2023
3,FULLA LUDOVIT,Ukrižovanie,1,106500.0,78000.0,okolo 1939-42,olej na kartóne,05. Október 2010,https://www.soga.sk/aukcie-obrazy-diela-umenie...,True,1939,2010
4,FOLTYN FRANTISEK,Dvaja,1,93000.0,NaN,1925,Olej na plátne,"13. November 2012, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...,True,1925,2012


# Analyza

## Top 50

### Top 50 najdrazsich autorov podla priemernej ceny

In [29]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_vydrazenych_od_2004, 
ROUND(avg(konecna_cena)) as priemerna_vydrazena_cena_EUR,
min(konecna_cena)as minimalna_vydrazena_cena_EUR,
max(konecna_cena)as maximalna_vydrazena_cena_EUR
--sum(konecna_cena)as celkovo_vydrazene_EUR
from df 
where True 
and konecna_cena is not null 
--and olej
group by 1 
having count(*)>5
order by 3
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]

,autor,pocet_vydrazenych_od_2004,priemerna_vydrazena_cena_EUR,minimalna_vydrazena_cena_EUR,maximalna_vydrazena_cena_EUR
0,FULLA LUDOVIT,18,19436.0,300.0,106500.0
1,BENKA MARTIN,33,16686.0,240.0,146000.0
2,LADISLAV MEDNYANSZKY,67,12052.0,200.0,85000.0
3,HALA JAN,33,11935.0,200.0,112000.0
4,JAN MUDROCH,9,10698.0,30.0,23000.0
5,ALEXANDER BAZOVSKY MILOS,25,10199.0,166.0,34000.0
6,JOZEF MOUSSON TEODOR,7,9479.0,4000.0,22000.0
7,ENDRE NEMES,6,8461.0,60.0,34000.0
8,ANTON JASUSCH,6,6523.0,2700.0,11618.0
9,KRIVOS RUDOLF,12,6343.0,2000.0,18000.0


### Top 50 najdrazsich autorov - oleje

In [30]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_vydrazenych_od_2004, 
ROUND(avg(konecna_cena)) as priemerna_vydrazena_cena_EUR,
min(konecna_cena)as minimalna_vydrazena_cena_EUR,
max(konecna_cena)as maximalna_vydrazena_cena_EUR
--sum(konecna_cena)as celkovo_vydrazene_EUR
from df 
where True 
and konecna_cena is not null 
and olej
group by 1 
having count(*)>5
order by 3
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]

,autor,pocet_vydrazenych_od_2004,priemerna_vydrazena_cena_EUR,minimalna_vydrazena_cena_EUR,maximalna_vydrazena_cena_EUR
0,LADISLAV MEDNYANSZKY,31,24421.0,3200.0,85000.0
1,BENKA MARTIN,24,22643.0,1162.0,146000.0
2,HALA JAN,19,18076.0,1992.0,112000.0
3,ALEXANDER BAZOVSKY MILOS,15,15465.0,2000.0,34000.0
4,MILAN PASTEKA,8,14604.0,1328.0,38000.0
5,JAN MUDROCH,7,13720.0,6639.0,23000.0
6,JOZEF MOUSSON TEODOR,6,10192.0,4000.0,22000.0
7,IMRO WEINER-KRAL,8,9073.0,450.0,20000.0
8,KOMPANEK VLADIMIR,17,8586.0,900.0,32000.0
9,LALUHA MILAN,15,7402.0,1050.0,17600.0


### Top 50 najdrazsich diel

In [31]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and olej
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,BENKA MARTIN,Z Detvy,146000.0,1932,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/125-vecerna-aukcia/benka-martin-z-detvy-62558
1,CYPRIAN MAJERNIK,KRASOJAZDKYNE,125000.0,1938,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/majernik-cyprian-krasojazdkyne-76381
2,HALA JAN,Nad Važcom,112000.0,1943,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/155-letna-aukcia/jan-hala-nad-vazcom-75730
3,FULLA LUDOVIT,Ukrižovanie,106500.0,1939,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/fulla-ludovit-ukrizovanie-24250
4,FOLTYN FRANTISEK,Dvaja,93000.0,1925,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/foltyn-frantisek-dvaja-32078
5,LADISLAV MEDNYANSZKY,Močaristá krajina,85000.0,1900,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/116-vecerna-aukcia/mednyanszky-ladislav-mocarista-krajina-50433
6,JAKOBY JULIUS,Akt ženy so zrkadlom,80000.0,1940,2017,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/134-vecerna-aukcia/jakoby-julius-akt-zeny-so-zrkadlom-65988
7,FULLA LUDOVIT,Májové ráno,80000.0,1950,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/fulla-ludovit-majove-rano-24249
8,JAKOBY JULIUS,Giza pri práci (Pri výzdobe gombíkov),69000.0,1952,2022,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/153-letna-aukcia-diel-vytvarneho-umenia/jakoby-julius-giza-pri-praci-pri-vyzdobe-gombikov-75298
9,CYPRIAN MAJERNIK,Utečenci,68000.0,1943,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/125-vecerna-aukcia/majernik-cyprian-utecenci-62590


### Top 50 najdrazsich diel - nie oleje

In [32]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
and not olej
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,CYPRIAN MAJERNIK,KRASOJAZDKYNE,125000.0,1938,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/majernik-cyprian-krasojazdkyne-76381
1,CORNELIS DE WAEL,BOJOVÁ SCÉNA,53110.0,None,2007,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/73-aukcia-umeleckeho-remesla-a-starozitnosti/cornelis-de-wael-bojova-scena-14029
2,,ROMÁNSKA TRÓNIACA MADONA S DIEŤAŤOM,51451.0,None,2007,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/73-aukcia-umeleckeho-remesla-a-starozitnosti/romanska-troniaca-madona-s-dietatom-14022
3,HALA JAN,Jar - Leto - Jeseň - Zima (36 ilustrácií),35000.0,1943,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/132-zimna-aukcia-vytvarnych-diel-a-starozitnosti/hala-jan-jar-leto-jesen-zima-36-ilustracii-65512
4,FULLA LUDOVIT,Tradícia - kartón mozaiky pre Svetovú výstavu v Montreale,30400.0,1967,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/98-letna-aukcia-vytvarnych-diel-a-starozitnosti/fulla-ludovit-tradicia-karton-mozaiky-pre-svetovu-vystavu-v-montreale-28242
5,CYPRIAN MAJERNIK,Žongléri,29000.0,1932,2013,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/113-vecerna-aukcia/majernik-cyprian-zongleri-48010
6,,PÁR STRIEBORNÝCH OLTÁRNYCH SVIETNIKOV,23236.0,None,2007,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/73-aukcia-umeleckeho-remesla-a-starozitnosti/par-striebornych-oltarnych-svietnikov-14028
7,FULLA LUDOVIT,Kartón pre mozaiku na bratislavskom hrade,20746.0,1968,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/81-jesenna-aukcia-vytvarnych-diel/ludovit-fulla-karton-pre-mozaiku-na-bratislavskom-hrade-18169
8,RUDOLF UHER,Kompozícia v čiernej a bielej farbe,20100.0,1966,2021,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/151-vianocna-aukcia/rudolf-uher-kompozicia-v-ciernej-a-bielej-farbe-69658
9,JULIUS KOLLER,PRECHODNÁ KULTÚRNA SITUÁCIA Č. 2 (ANTIOBRAZ),19500.0,1990,2024,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/158-zimna-aukcia/koller-julius-prechodna-kulturna-situacia-c-2-antiobraz-76269


### Top 50 najdrazsich diel - Pasteka

In [33]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%PASTEKA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,MILAN PASTEKA,V izbe,38000.0,1975,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/91-aukcia-sucasneho-umenia/pasteka-milan-v-izbe-21937
1,MILAN PASTEKA,Zátišie,24000.0,1974,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/pasteka-milan-zatisie-32098
2,MILAN PASTEKA,Spojenie,17000.0,1989,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/155-letna-aukcia/milan-pasteka-spojenie-75775
3,MILAN PASTEKA,Objatie,14100.0,1987,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/pasteka-milan-objatie-24012
4,MILAN PASTEKA,Zátišie s jablkami a hruškou,9000.0,1961,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/156-zimna-aukcia/milan-pasteka-zatisie-s-jablkami-a-hruskou-75931
5,MILAN PASTEKA,Čakanie,7400.0,1982,2009,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/84-jarna-aukcia-vytvarnych-diel/pasteka-milan-cakanie-18747
6,MILAN PASTEKA,Dialóg,7300.0,1966,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/155-letna-aukcia/milan-pasteka-dialog-75721
7,MILAN PASTEKA,Zátišie so džbánom,6000.0,1970,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/pasteka-milan-zatisie-so-dzbanom-24011
8,MILAN PASTEKA,DIVÁCI,4700.0,1969,2024,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/158-zimna-aukcia/pasteka-milan-divaci-76273
9,MILAN PASTEKA,ROZHOVOR,3900.0,1987,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/pasteka-milan-rozhovor-76376


### Top 50 najdrazsich diel - Binder

In [34]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%BINDER %'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,BINDER ERIK,NÁMORNÍKOVA OPICA A OPICA,5500.0,2019,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/binder-erik-namornikova-opica-a-opica-76464
1,BINDER ERIK,"Ja somár, načo som sa snažil?!",5000.0,2014,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/binder-erik-ja-somar-naco-som-sa-snazil-52982
2,BINDER ERIK,Očorúčne reči.,1500.0,2011,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/103-aukcia-sucasneho-umenia/binder-erik-1974-paulovicova-barbora-ocorucne-reci-30089
3,BINDER ERIK,"Ty, had!",66.0,2004,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/50-mimoriadna-aukcia-sucasneho-slovenskeho-umenia/erik-binder-ty-had-5598


### Top 50 najdrazsich diel - Filko

In [35]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FILKO%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,FILKO STANO,Modrý objekt II.,6000.0,1966,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/filko-stano-modry-objekt-ii-32123
1,FILKO STANO,Hlavy,3600.0,1957,2009,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/85-aukcia-sucasneho-slovenskeho-umenia/filko-stano-hlavy-19325
2,FILKO STANO,Proces,1500.0,1962,2019,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/146-aukcia-vytvarnych-diel/filko-stano-proces-69019
3,FILKO STANO,Asociácie XX.,465.0,1969,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/68-zimna-aukcia-vytvarnych-diel/stano-filko-asociacie-xx-12489
4,FILKO STANO,Asociácie XXXVII.,266.0,1969,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/68-zimna-aukcia-vytvarnych-diel/stano-filko-asociacie-xxxvii-12488
5,FILKO STANO,Komplex asociácií - A,266.0,1970,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/68-zimna-aukcia-vytvarnych-diel/stano-filko-komplex-asociacii-a-12487


### Top 50 najdrazsich diel - Laluha

In [36]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%LALUHA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,LALUHA MILAN,Kytica,17600.0,1959,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/129-letna-aukcia-vytvarnych-diel-a-starozitnosti/laluha-milan-kytica-63514
1,LALUHA MILAN,Akt,15700.0,1970,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/laluha-milan-akt-52950
2,LALUHA MILAN,CHLAP SO ŽENOU II,15000.0,1968,2024,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/158-zimna-aukcia/laluha-milan-chlap-so-zenou-ii-76329
3,LALUHA MILAN,Hrabáči z Mičinej,13200.0,1965,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/91-aukcia-sucasneho-umenia/laluha-milan-hrabaci-z-micinej-21900
4,LALUHA MILAN,Tetka,11000.0,1975,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/95-zimna-aukcia-vytvarnych-diel-a-starozitnosti/laluha-milan-tetka-25419
5,LALUHA MILAN,Žena z Mičine,7200.0,1975,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/155-letna-aukcia/milan-laluha-zena-z-micine-75760
6,LALUHA MILAN,Žena s nošou,5800.0,1965,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/156-zimna-aukcia/milan-laluha-zena-s-nosou-75865
7,LALUHA MILAN,Zamyslená,5100.0,1975,2017,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/138-zimna-aukcia-vytvarnych-diel-a-starozitnosti/laluha-milan-zamyslena-66892
8,LALUHA MILAN,Chlap s hrabľami.,5000.0,1975,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/103-aukcia-sucasneho-umenia/laluha-milan-chlap-s-hrablami-30083
9,LALUHA MILAN,Strechy z Mičinej,4000.0,1968,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/laluha-milan-strechy-z-micinej-24017


### Top 50 najdrazsich diel - Benka

In [37]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%BENKA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,BENKA MARTIN,Z Detvy,146000.0,1932,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/125-vecerna-aukcia/benka-martin-z-detvy-62558
1,BENKA MARTIN,Cestou (Ružomberok),63000.0,1931,2017,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/134-vecerna-aukcia/benka-martin-cestou-ruzomberok-65984
2,BENKA MARTIN,NÁČRT Z LIPTOVA,47000.0,1935,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/benka-martin-nacrt-z-liptova-76394
3,BENKA MARTIN,Pod Chočom,42000.0,1921,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/benka-martin-pod-chocom-52918
4,BENKA MARTIN,V nedeľu (Vlkolinec),42000.0,1920,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/98-letna-aukcia-vytvarnych-diel-a-starozitnosti/benka-martin-v-nedelu-vlkolinec-28254
5,BENKA MARTIN,Z Liptova (Ružomberok),25000.0,1935,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/benka-martin-z-liptova-ruzomberok-52906
6,BENKA MARTIN,Štúdia – Revúca,25000.0,1925,2022,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/154-zimna-aukcia-vytvarnych-diel/benka-martin-studia-revuca-75446
7,BENKA MARTIN,Malé Karpaty (Od Kuchyne),21000.0,1930,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/115-jarna-aukcia-vytvarnych-diel/benka-martin-male-karpaty-od-kuchyne-50055
8,BENKA MARTIN,KRAJINA NA VÁHU – TEMATÍN,17000.0,1932,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/benka-martin-krajina-na-vahu-tematin-76358
9,BENKA MARTIN,V DEDINE,15000.0,1918,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/benka-martin-v-dedine-76440


### Top 50 najdrazsich diel - Fulla

In [38]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FULLA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,FULLA LUDOVIT,Ukrižovanie,106500.0,1939,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/fulla-ludovit-ukrizovanie-24250
1,FULLA LUDOVIT,Májové ráno,80000.0,1950,2010,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/93-jesenna-aukcia-vytvarnych-diel/fulla-ludovit-majove-rano-24249
2,FULLA LUDOVIT,Kytica cyklámenov,56430.0,1930,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/81-jesenna-aukcia-vytvarnych-diel/ludovit-fulla-kytica-cyklamenov-18170
3,FULLA LUDOVIT,Tradícia - kartón mozaiky pre Svetovú výstavu v Montreale,30400.0,1967,2011,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/98-letna-aukcia-vytvarnych-diel-a-starozitnosti/fulla-ludovit-tradicia-karton-mozaiky-pre-svetovu-vystavu-v-montreale-28242
4,FULLA LUDOVIT,Jánošík,21000.0,1956,2019,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/147-vecerna-aukcia/fulla-ludovit-janosik-69135
5,FULLA LUDOVIT,Kartón pre mozaiku na bratislavskom hrade,20746.0,1968,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/81-jesenna-aukcia-vytvarnych-diel/ludovit-fulla-karton-pre-mozaiku-na-bratislavskom-hrade-18169
6,FULLA LUDOVIT,Kostol Všetkých svätých pri Ružomberku,16597.0,1949,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/68-zimna-aukcia-vytvarnych-diel/ludovit-fulla-kostol-vsetkych-svatych-pri-ruzomberku-12638
7,FULLA LUDOVIT,Toaleta,4900.0,1961,2022,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/154-zimna-aukcia-vytvarnych-diel/fulla-ludovit-toaleta-75435
8,FULLA LUDOVIT,"Päť ilustrácii ku knihe ""Spevník...""",4000.0,1964,2018,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/142-jesenna-aukcia-vytvarnych-diel/ludovit-fulla-pat-ilustracii-ku-knihe-spevnik-67969
9,FULLA LUDOVIT,Zvestovanie,4000.0,1942,2023,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/156-zimna-aukcia/ludovit-fulla-zvestovanie-75857


### Top 50 najdrazsich diel - Klimo

In [39]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%KLIMO%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,ALOJZ KLIMO,NADHĽAD,5500.0,1967,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/klimo-alojz-nadhlad-76370
1,ALOJZ KLIMO,KRIŽOVATKA,3800.0,1966,2024,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/158-zimna-aukcia/klimo-alojz-krizovatka-76263
2,ALOJZ KLIMO,Križovatka,2500.0,1967,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/132-zimna-aukcia-vytvarnych-diel-a-starozitnosti/klimo-alojz-krizovatka-65668
3,ALOJZ KLIMO,Križovatka,1195.0,1975,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/80-letna-aukcia-vytvarnych-diel-a-starozitnosti/alojz-klimo-krizovatka-17092
4,ALOJZ KLIMO,Štvorce,1162.0,1992,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/68-zimna-aukcia-vytvarnych-diel/alojz-klimo-stvorce-12479
5,ALOJZ KLIMO,Vertikála,1050.0,1996,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/129-letna-aukcia-vytvarnych-diel-a-starozitnosti/klimo-alojz-vertikala-63545
6,ALOJZ KLIMO,Štvorce,1050.0,None,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/klimo-alojz-stvorce-76341
7,ALOJZ KLIMO,Konštruktivistická kompozícia,240.0,1980,2009,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/85-aukcia-sucasneho-slovenskeho-umenia/klimo-alojz-konstruktivisticka-kompozicia-19215
8,ALOJZ KLIMO,Okno,230.0,1970,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/120-zimna-aukcia-vytvarnych-diel-a-starozitnosti/klimo-alojz-okno-52356
9,ALOJZ KLIMO,Kompozícia v červenom,190.0,1975,2009,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/85-aukcia-sucasneho-slovenskeho-umenia/klimo-alojz-kompozicia-v-cervenom-19213


### Top 50 najdrazsich diel - Roller

In [40]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%ROLLER%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,PETER ROLLER,Kameň,1162.0,None,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/56-aukcia-sucasneho-slovenskeho-umenia/peter-roller-kamen-8119
1,PETER ROLLER,Svetlo monolitu.,350.0,2004,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/103-aukcia-sucasneho-umenia/roller-peter-svetlo-monolitu-30090


### Top 50 najdrazsich diel - Fila

In [41]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FILA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,FILA RUDOLF,Pokus o úlet,9000.0,1993,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/119-vecerna-aukcia/fila-rudolf-pokus-o-ulet-51774
1,FILA RUDOLF,Za šrafami,7000.0,1967,2022,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/154-zimna-aukcia-vytvarnych-diel/fila-rudolf-za-srafami-75507
2,FILA RUDOLF,Laokonka,4500.0,1991,2012,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/107-vecerna-aukcia/fila-rudolf-laokonka-32112
3,FILA RUDOLF,KNIHA O FRESKÁCH I,3800.0,1995,2025,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/159-letna-aukcia/fila-rudolf-kniha-o-freskach-i-76397
4,FILA RUDOLF,Načúvanie - Divertimento s citátom,3319.0,1987,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/56-aukcia-sucasneho-slovenskeho-umenia/rudolf-fila-nacuvanie-divertimento-s-citatom-8196
5,FILA RUDOLF,Za clonou,2600.0,1972,2013,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/111-letna-aukcia-vytvarnych-diel-a-starozitnosti/fila-rudolf-za-clonou-45803
6,FILA RUDOLF,ČO VIDÍME II,2500.0,2001,2024,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/158-zimna-aukcia/fila-rudolf-co-vidime-ii-76288
7,FILA RUDOLF,Zápalné,2000.0,2004,2016,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/129-letna-aukcia-vytvarnych-diel-a-starozitnosti/fila-rudolf-zapalne-63528
8,FILA RUDOLF,Do priestoru,1925.0,1999,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/56-aukcia-sucasneho-slovenskeho-umenia/rudolf-fila-do-priestoru-8195
9,FILA RUDOLF,Pocta Messerschmidtovi II.,1328.0,1983,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/68-zimna-aukcia-vytvarnych-diel/rudolf-fila-pocta-messerschmidtovi-ii-12480


## Najvyssi narast a pokles cien

### Najvyssi narast

In [42]:
from scipy.stats import linregress
pd.set_option('display.max_colwidth', None)

# Calculate the average price for each name in each year
df_avg = df.groupby(['autor', 'rok_drazby'])['konecna_cena'].agg(['count', 'mean']).reset_index()
df_avg = df_avg[df_avg['count']>=3][['autor', 'rok_drazby', 'mean']]

# Filter authors with at least 50 rows
author_counts = df_avg['autor'].value_counts()
selected_authors = author_counts[author_counts >= 5].index
df_avg_filtered = df_avg[df_avg['autor'].isin(selected_authors)]

# Calculate the slope for each author
slopes = []

for author in selected_authors:
    author_data = df_avg_filtered[df_avg_filtered['autor'] == author]
    slope, _, _, _, _ = linregress(author_data['rok_drazby'], author_data['mean'])
    slopes.append({'autor': author, 'priemerny_rocny_narast_v_cene': slope})

# Create a DataFrame for the slopes
slopes_df = pd.DataFrame(slopes)

# Display the slopes
slopes_df[slopes_df['priemerny_rocny_narast_v_cene'].notna()].sort_values('priemerny_rocny_narast_v_cene', ascending=False).reset_index(drop=True)[0:20]

ModuleNotFoundError: No module named 'scipy'

In [ ]:
slopes_df[slopes_df['priemerny_rocny_narast_v_cene'].notna()].sort_values('priemerny_rocny_narast_v_cene', ascending=True).reset_index(drop=True)[0:20]

In [ ]:
# Calculate the average price for each name in each year
df_avg = df.groupby(['autor', 'rok_drazby'])['konecna_cena'].mean().reset_index()

# Filter authors with at least 50 rows
selected_authors = ['MILAN PASTEKA', 'FILKO STANO', 'LALUHA MILAN']
df_avg_filtered = df_avg[df_avg['autor'].isin(selected_authors)]

# Create Bokeh ColumnDataSource
source = ColumnDataSource(df_avg_filtered)

# Plotting
output_notebook()
p = figure(plot_width=800, plot_height=400, title='Average Price Trends Over Time for Each Name',
           x_axis_label='Year', y_axis_label='Average Price')

# Add a line for each autor
for color, autor in zip(Category10[6], df_avg_filtered['autor'].unique()):
    autor_data = df_avg_filtered[df_avg_filtered['autor'] == autor]
    autor_source = ColumnDataSource(autor_data)
    p.line(x='rok_drazby', y='konecna_cena', source=autor_source, line_width=2, color=color, legend_label=autor)
    p.circle(x='rok_drazby', y='konecna_cena', source=autor_source, size=8, color=color, legend_label=autor)

p.legend.location = 'top_left'
show(p)

## Najvyssie percento vydrazenych

In [ ]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_drazenych_od_2004, 
sum(vydrazene) as pocet_vydrazenych_od_2004,
round(sum(vydrazene)*100.0/count(*),2) as percento_vydrazenych
from df 
where True 
--and konecna_cena is not null 
--and olej
group by 1 
having count(*)>50
order by 4
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]